# Examples and features

Here we will showcase different types of problems that the package can solve. The examples can also be downloaded as a Jupyter notebook from the [GitHub repository](https://github.com/ecboghiu/inflation).

We begin by importing the functions that we will need:

In [1]:
from causalinflation import InflationProblem, InflationSDP
from examples_utils import bisection, P_W_array, P_2PR_array, P_PRbox_array

## Feasibility problems and extraction of certificates

### Example 1: Infeasibility of the W distribution in the quantum triangle scenario 

Consider determining if the following distribution, the so-called "W distribution" (due to its similarity to the [W state](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.62.062314)), is compatible with the triangle scenario:

$$ P_{A B C}=\frac{[100]+[010]+[001]}{3}, \quad \text {i.e.,} \quad P_{A B C}(a b c)= \begin{cases}\frac{1}{3} & \text { if } a+b+c=1, \\ 0 & \text { otherwise. }\end{cases} $$

It is known that it is [incompatible with the classical triangle scenario](https://www.degruyter.com/document/doi/10.1515/jci-2017-0020/html), however with quantum inflation, once can also show that it is incompatible with the quantum triangle scenario, depicted in the following figure:

<center> <img src="./figures/quantum_triangle.PNG" alt="drawing" width="250"/> </center>

To show this, we can generate the semidefinite relaxation of NPA level 2 corresponding to a second order quantum inflation:

In [2]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"]}, 
                             outcomes_per_party=[2, 2, 2],
                             settings_per_party=[1, 1, 1],
                             inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle)
sdprelax.generate_relaxation('npa2')

With the `set_distribution` method we set the entries of the moment matrix that depend on the probability distribution, and we attempt to solve the program with the `solve()` method:

In [3]:
sdprelax.set_distribution(P_W_array)
sdprelax.solve()
sdprelax.status

'infeasible'

The problem status is reported as infeasible, therefore this serves as a proof that the W distribution is incompatible with the quantum triangle scenario. 

#### Certificate extraction

We can furthermore recover a certificate of infeasibility as a polynomial inequality in the probabilities, $\text{Poly}(p(abc|xyz) \geq 0$ . This means that any other correlations vector $p'(abc|xyz)$ that also satisfies the inequality, $\text{Poly}(p'(abc|xyz) \geq 0$, is guaranteed to lead to an infeasible SDP. There are built-in methods to extract the symbolic form of $\text{Poly}(p(abc|xyz))$:

In [4]:
sdprelax.certificate_as_probs(clean=True)

0.476*p(000|000)*p_{A}(0|0) + 0.476*p(000|000)*p_{B}(0|0) + 0.476*p(000|000)*p_{C}(0|0) + 0.75*p(000|000) + 0.217*p_{AB}(00|00)**2 + 0.085*p_{AB}(00|00)*p_{AC}(00|00) + 0.059*p_{AB}(00|00)*p_{A}(0|0) + 0.085*p_{AB}(00|00)*p_{BC}(00|00) + 0.059*p_{AB}(00|00)*p_{B}(0|0) - 0.359*p_{AB}(00|00)*p_{C}(0|0) + 0.126*p_{AB}(00|00) + 0.217*p_{AC}(00|00)**2 + 0.059*p_{AC}(00|00)*p_{A}(0|0) + 0.085*p_{AC}(00|00)*p_{BC}(00|00) - 0.359*p_{AC}(00|00)*p_{B}(0|0) + 0.059*p_{AC}(00|00)*p_{C}(0|0) + 0.126*p_{AC}(00|00) - 0.476*p_{A}(0|0)**2 - 0.359*p_{A}(0|0)*p_{BC}(00|00) + 1.0*p_{A}(0|0)*p_{B}(0|0)*p_{C}(0|0) - 0.765*p_{A}(0|0)*p_{B}(0|0) - 0.765*p_{A}(0|0)*p_{C}(0|0) + 0.365*p_{A}(0|0) + 0.217*p_{BC}(00|00)**2 + 0.059*p_{BC}(00|00)*p_{B}(0|0) + 0.059*p_{BC}(00|00)*p_{C}(0|0) + 0.126*p_{BC}(00|00) - 0.476*p_{B}(0|0)**2 - 0.765*p_{B}(0|0)*p_{C}(0|0) + 0.365*p_{B}(0|0) - 0.476*p_{C}(0|0)**2 + 0.365*p_{C}(0|0) + 0.563

In the above, lower indices indicate marginals. For example, $p_{AC}(ac|xz) := \sum_b p(abc|xyz)$. Note that due to no-signaling, in this example the marginal is independent of the setting $y$.

 Finally, given that we only have two outcomes, we can also express the certificate in "correlator form", where the correlators are defined as 
 
 $$\left\langle A_{x} \right\rangle =\sum_{a\in \{0,1\}} (-1)^{a} \, p_{A}(a|x)$$ 
 $$\left\langle A_{x} B_{y} \right\rangle =\sum_{a, b \in \{0,1\}} (-1)^{a+b} \, p_{AB}(ab|xy)$$
 $$\left\langle A_{x} B_{y} C_{z}\right\rangle =\sum_{a, b, c \in \{0,1\}} (-1)^{a+b+c} \, p(abc|xyz)$$ 
 
 where the omitted 2-body and 1-body correlators have similar definitions:

In [5]:
sdprelax.certificate_as_correlators(clean=True)

0.02975*\langle A_{0} B_{0} C_{0} \rangle*\langle A_{0} \rangle + 0.02975*\langle A_{0} B_{0} C_{0} \rangle*\langle B_{0} \rangle + 0.02975*\langle A_{0} B_{0} C_{0} \rangle*\langle C_{0} \rangle - 0.183*\langle A_{0} B_{0} C_{0} \rangle + 0.0135625*\langle A_{0} B_{0} \rangle**2 + 0.0053125*\langle A_{0} B_{0} \rangle*\langle A_{0} C_{0} \rangle - 0.0695625*\langle A_{0} B_{0} \rangle*\langle A_{0} \rangle + 0.0053125*\langle A_{0} B_{0} \rangle*\langle B_{0} C_{0} \rangle - 0.0695625*\langle A_{0} B_{0} \rangle*\langle B_{0} \rangle + 0.0045*\langle A_{0} B_{0} \rangle*\langle C_{0} \rangle + 0.222125*\langle A_{0} B_{0} \rangle + 0.0135625*\langle A_{0} C_{0} \rangle**2 - 0.0695625*\langle A_{0} C_{0} \rangle*\langle A_{0} \rangle + 0.0053125*\langle A_{0} C_{0} \rangle*\langle B_{0} C_{0} \rangle + 0.0045*\langle A_{0} C_{0} \rangle*\langle B_{0} \rangle - 0.0695625*\langle A_{0} C_{0} \rangle*\langle C_{0} \rangle + 0.222125*\langle A_{0} C_{0} \rangle - 0.0420625*\langle A_{0} \r

### Example 2: Critical visibility of the 2PR distribution in the quantum tripartite-line scenario

It is known that the 2PR distribution, defined as:

$$ P_{\text{2PR}}(abc|xyz) := \frac{1+ (-1)^{a+b+c+xy+yz}}{8} $$

is incompatible with the tripartite-line scenario (also called "quantum bilocal scenario"), whose DAG is depicted in the following figure:

<center> <img src="./figures/bilocal_1.PNG" alt="drawing" width="400"/> </center>

This can be shown by running a feasibility program, as in Example 1. We might also be interested in studying how much noise this distribution can tolerate before the relaxation no longer identifies the distribution as incompatible. One simple model of noise is that of a probabilistic mixture with the uniform distribution:

$$ P_{\text{2PR,v}} := v P_{\text{2PR}}  + (1-v)/8 $$

A simple approach would be to vary the parameter $v$ from $v{=}1$ to $v{=}0$ and find the $v_{\text{crit}}$ for which the problem status changes from infeasible to feasible. However, there is a more robust method available.

#### Feasibility as an optimisation

A more numerically robust approach is to convert feasibility problems to optimisation problems. Instead of imposing that the moment matrix $\Gamma$ of the SDP relaxation is positive semidefinite, we can maximize the minimum eigenvalue of $\Gamma$ and check its sign. Clearly, if the result of the optimisation is negative, then one cannot find a matrix $\Gamma$ that is positive semidefinite, thus the original program is infeasible. 

By setting the flag `feas_as_optim` to `True` in the `InflationSDP.solve()` method, feasibility problems are converted to optimisation problems. The result is stored in `InflationSDP.objective_value`. 

We encode the inflation scenario and generate the relaxation corresponding to NPA level 2:

In [6]:
qbilocal = InflationProblem(dag={"rho_AB": ["A", "B"],
                                 "rho_BC": ["B", "C"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2])
qbilocal_relax = InflationSDP(qbilocal)
qbilocal_relax.generate_relaxation('npa2')

And next we run a simple bisection to find the $v_{\text{crit}}$ for which the maximum minimum eigenvalue is 0. The bisection is implemented in an auxiliary file:

In [7]:
sdp, v_crit = bisection(qbilocal_relax, P_2PR_array)
v_crit

0.500030517578125

We recover a critical visibility that is consistent with the known critical visibility of the 2PR distribution in the quantum tripartite-line scenario, namely $v>\frac{1}{2}$. Higher order inflations or higher levels in the NPA hierarchy are expected (but not proved) to get a numerical visibility that gets asymptotically closer to $v=\frac{1}{2}$ as the hierarchy increases.

## Optimization of Bell operators

One can use inflation techniques to not only run causal compatibility problems, but also to optimize over the generated relaxation, and therefore get upper bounds on the values of various Bell operators.

### Example 3. Upper bounds on Mermin's inequality

Let us consider Mermin's inequality, written in the correlator form introduced in Example 1:

$$ \text{Mermin} = \langle A_1 B_0 C_0 \rangle +  \langle A_0 B_1 C_0 \rangle +  \langle A_0 B_0 C_1 \rangle -  \langle A_1 B_1 C_1 \rangle $$

It is known that the algebraic maximum of 4 is achieved in the tripartite scenario both with global shared randomness and also global non-signaling sources. However, one can see a difference between quantum and general no-signaling sources when restricting to the triangle scenario from Example 1.

First we generate the relaxation corresponding to a second order inflation of the triangle of NPA level 2: 

In [20]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"]}, 
                             outcomes_per_party=[2, 2, 2],
                             settings_per_party=[2, 2, 2],
                             inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle)
sdprelax.generate_relaxation('npa2')

We implement the objective function after extracting the measurement operators and solve the program:

In [21]:
mmnts = sdprelax.measurements
A0, B0, C0, A1, B1, C1 = (1-2*mmnts[party][0][setting][0] for setting in range(2) for party in range(3))

sdprelax.set_objective(objective = A1*B0*C0 + A0*B1*C0 + A0*B0*C1 - A1*B1*C1)
sdprelax.solve()

sdprelax.objective_value

3.999999981256284

Notice that we get a value that is within numerical precision the algebraic maximum of 4. To improve on this result, we will need to do a tighter SDP relaxation.

#### Customising the generating set for the semidefinite relaxation

To get a tighter SDP relaxation, we will add more monomoials to the generating set. Namely, we will use the union of the monomoials corresponding to NPA level 2 and local level 1.

The so-called "local levels" are a different choice of generating set for the moment matrix. Whereas NPA level $n$ is the $n$-times cartesian product (without duplicated elements) of the set of measurements of the parties together with the identity, local level $n$ refers to a generating set with all the products up to $n$ operators per party. For more details, see [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043).

In what follows, we use the built-in method `InflationSDP.build_columns()` to generate the columns corresponding to NPA level 2 and local level 1. Then we do a union, generate the relaxation and again, solve the program. As it will now take a bit longer, we increase the verbosity level to see the progress: 

In [22]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"]}, 
                             outcomes_per_party=[2, 2, 2],
                             settings_per_party=[2, 2, 2],
                             inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle)
# sdprelax.generate_relaxation('npa2')

npa2   = sdprelax.build_columns('npa2')
local1 = sdprelax.build_columns('local1')

newgeneratingset = npa2
for mon in local1:
     if mon not in newgeneratingset:
         newgeneratingset.append(mon)

sdprelax.generate_relaxation(newgeneratingset)
sdprelax.set_objective(objective = A1*B0*C0 + A0*B1*C0 + A0*B0*C1 - A1*B1*C1)

sdprelax.verbose = 1
sdprelax.solve()
sdprelax.objective_value

Problem
  Name                   : InflationSDP    
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 17840           
  Cones                  : 0               
  Scalar variables       : 1904            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : InflationSDP    
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

3.0850445229084524

After running the above, we can certify then that the Mermin inequality cannot have a value larger than $3.085$ for the quantum triangle causal scenario. 

## Optimisation over classical distributions and feasibility problems 

With quantum inflation, we can also optimize over a relaxation of the set of distributions compatible with a classical DAG. This works by imposing at the level of the SDP relaxation the constraint that all operators defining the moments commute. The effect of this constraint is that previously different variables in the moment matrix become identical. For example, $\langle A_{x} A_{x'} \rangle \neq \langle A_{x'} A_{x} \rangle$ in general in quantum mechanics, but if we assume all operators commute, then they become equal. 

To enable this feauture one simply adds the flag `commuting=True` when instantiating the `InflationSDP` object. 

### Example 4: Critical visibility of the 2PR distribution in the *classical* tripartite-line scenario

As an example, we find the critical visibility of the $P_{\text{2PR}}$ distribution from Example 2, but in the classical tripartite line scenario with a second order inflation, and with the local level 1 generating set for the SDP relaxation (local levels are introduced in Example 3):

In [11]:
qbilocal = InflationProblem(dag={"rho_AB": ["A", "B"], "rho_BC": ["B", "C"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2])
qbilocal_relax = InflationSDP(qbilocal, commuting=True)
qbilocal_relax.generate_relaxation('local1')
sdp, v_crit = bisection(qbilocal_relax, P_2PR_array, verbose=0)
v_crit

0.353546142578125

This relaxation of the set of distributions classically simulable in the tripartite line scenario certifies then the incompatibility of the $P_{\text{2PR}}$ distribution for $v>0.3536$. This does not completely certify incompatibility down to the known critical threshold of $v_{\text{crit}}=\frac{1}{4}$, but we expect tighter relaxations, which are computationally more expensive, might recover this value. 

For optimisation problems, one can run the exact same program as in Example 3, but with the flag `commuting` set to `True`.

## Standard NPA

If the DAG corresponds to a single global shared source scenario, then doing an inflation does not grant any advantage. In this case, the semidefinite programming relaxation defaults to being the same as the [NPA hierarcy](https://iopscience.iop.org/article/10.1088/1367-2630/10/7/073013). If we set the `commuting` flag to `True` then this is a relaxation of the set of distributions classically with global shared randomness, as introduced in [Phys. Rev. X 7, 021042](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.021042). We will show this with two simple examples.

### Example 5: Critical visibility of the PR box in the standard Bell scenario with quantum sources

We recover the critical visibility of $v_{\text{crit}}=\frac{1}{\sqrt{2}}$ for a [Popeschu-Rohrlich box](https://link.springer.com/article/10.1007/BF02058098) in the Bell scenario.  

In [12]:
bellscenario = InflationProblem(dag={"rho_AB": ["A", "B"]},
                                outcomes_per_party=[2, 2], 
                                settings_per_party=[2, 2])
bellscenario_relax = InflationSDP(bellscenario)
bellscenario_relax.generate_relaxation('npa1')
sdp, v_crit = bisection(bellscenario_relax, P_PRbox_array, verbose=1)
print("Critical visibility: ", v_crit)
sdp.certificate_as_correlators(clean=True)*-4

0 Maximum smallest eigenvalue:    0.06623 	visibility = 0.5
1 Maximum smallest eigenvalue:   -0.01418 	visibility = 0.75
2 Maximum smallest eigenvalue:    0.02683 	visibility = 0.625
3 Maximum smallest eigenvalue:   0.006449 	visibility = 0.6875
4 Maximum smallest eigenvalue:  -0.003839 	visibility = 0.7188
5 Maximum smallest eigenvalue:   0.001311 	visibility = 0.7031
6 Maximum smallest eigenvalue:  -0.001262 	visibility = 0.7109
7 Maximum smallest eigenvalue:  2.489e-05 	visibility = 0.707
8 Maximum smallest eigenvalue: -0.0006186 	visibility = 0.709
9 Maximum smallest eigenvalue: -0.0002968 	visibility = 0.708
10 Maximum smallest eigenvalue:  -0.000136 	visibility = 0.7075
11 Maximum smallest eigenvalue: -5.554e-05 	visibility = 0.7073
12 Maximum smallest eigenvalue: -1.533e-05 	visibility = 0.7072
13 Maximum smallest eigenvalue:   4.78e-06 	visibility = 0.7071
Critical visibility:  0.707122802734375


\langle A_{0} B_{0} \rangle + \langle A_{0} B_{1} \rangle + \langle A_{1} B_{0} \rangle - 1.0*\langle A_{1} B_{1} \rangle - 2.828

Notice that the dual certificate that we extract in correlator form (which has been renormalised and rounded numerically) is the CHSH inequality tangent to the quantum set of correlations.

### Example 6: Optimising the CHSH inequality in the Bell scenario

To find upper bounds on the values for the CHSH Bell operator, recovered in Example 5, we run a similar program to that in Example 2:

In [13]:
bellscenario_relax = InflationSDP(bellscenario)
bellscenario_relax.generate_relaxation('npa1')

mmnts = bellscenario_relax.measurements
A0, B0, A1, B1 = (1-2*mmnts[party][0][x][0] for x in [0, 1] for party in [0, 1])

bellscenario_relax.set_objective(A0*B0 + A0*B1 + A1*B0 - A1*B1)
bellscenario_relax.solve()
bellscenario_relax.objective_value

2.8284271429557952

We correctly recover the Tsirelson bound for the CHSH inequality.

## SDP hierarchy of "physical moments"

For the generation of the semidefinite programming relaxation, besides NPA levels and local levels, we also implement a hierachy of "physical moments" of level $n$. This is a subset of local level $n$ of all the monomials for which all operators in that monomial commute due to non-overlapping support in the inflated graph.

### Example 7: Critical visibility of the W distribution with the physical moments hierarchy

 As an application, we show how we can recover the critical visibility $v_{\text{crit}}\approx=0.8038$ of the W distribution in the triangle causal scenario achieved with the generating set corresponding to local level 2 with monomials of maximum length 4, as shown in [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043). This corresponds to a moment matrix of size 1175x1175. However, by using the second level of the physical moments hierarchy of monomials up to length 4, we recover the same results with a smaller moment matrix:

In [14]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"],},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[1, 1, 1],
                            inflation_level_per_source=[2, 2, 2])
qtriangle_relax = InflationSDP(qtriangle)
cols = qtriangle_relax.build_columns('physical2', max_monomial_length=4)
qtriangle_relax.generate_relaxation(cols)

sdp, v_crit = bisection(qtriangle_relax, P_W_array, verbose=1)
v_crit

0 Maximum smallest eigenvalue:  0.0001523 	visibility = 0.5
1 Maximum smallest eigenvalue:  4.132e-05 	visibility = 0.75
2 Maximum smallest eigenvalue:  -0.003502 	visibility = 0.875
3 Maximum smallest eigenvalue:  -0.000197 	visibility = 0.8125
4 Maximum smallest eigenvalue:  3.192e-05 	visibility = 0.7812
5 Maximum smallest eigenvalue:  2.283e-05 	visibility = 0.7969
6 Maximum smallest eigenvalue: -1.558e-05 	visibility = 0.8047
7 Maximum smallest eigenvalue:  1.074e-05 	visibility = 0.8008
8 Maximum smallest eigenvalue:  3.997e-06 	visibility = 0.8027
9 Maximum smallest eigenvalue:   4.96e-07 	visibility = 0.8037
10 Maximum smallest eigenvalue: -6.422e-06 	visibility = 0.8042
11 Maximum smallest eigenvalue: -1.948e-06 	visibility = 0.804
12 Maximum smallest eigenvalue:  5.734e-08 	visibility = 0.8038
13 Maximum smallest eigenvalue: -8.405e-07 	visibility = 0.8039


0.803863525390625

We recover the same critical visibility of $v_{\text{crit}}\approx 0.8039$ a moment matrix of size 287x287 as opposed to 1175x1175, which leads to a signfificant gain in performance.

## Linearized polynomial identification (LPI) constraints

LPI constraints were introduced in [arXiv:2203.16543](https://arxiv.org/abs/2203.16543). They are proportionality constraints between different entries of the moment matrix. Due to nature of the inflated graph, many of the moments in the moment matrix factorise into products of other moments. 

As a simplified example, consider the moment $\langle A^{110}_{xa} B^{202}_{x'a'} B^{201}_{yb} \rangle$ in the inflated triangle of order two. In the triangle, we have three sources, $\rho_{AB}$, $\rho_{AC}$ and $\rho_{BC}$. The upper indices in the operators of the previous moment indicate on which copy of the sources the operator is acting. The value 0 means that the party does not measure the corresponding source. For example, $B^{201}_{yb}$ represents Bob measuring outcome $b$ of setting $y$ on copy 2 of $\rho_{AB}$ and copy 1 of $\rho_{BC}$. Notice that because of the non-overlapping support of some of the moments, the moment factorises as follows:

$$\langle A^{110}_{xa} B^{202}_{x'a'} B^{201}_{yb} \rangle = \langle A^{110}_{xa} \rangle \langle B^{202}_{x'a'} B^{201}_{yb} \rangle $$ 

The moment $\langle A^{110}_{xa} \rangle$ is known to be equal to $p_A(a|x)$, but  $\langle B^{202}_{x'a'} B^{201}_{yb} \rangle$ is unknown. Therefore, we have a linear proportionality relationship between the variables $\langle A^{110}_{xa} B^{202}_{x'a'} B^{201}_{yb} \rangle$ and $\langle B^{202}_{x'a'} B^{201}_{yb} \rangle$.

Proportionality constraints of these form can be automatically implemented by setting `use_lpi_constraints` to `True` when using the `set_distribution()` method.

### Example 8: Critical visibility of the W distribution with the physical moments hierarchy and LPI constraints

We will now show how using LPI constraints lead to tighter relaxations. For example, we can certify incompatiblity with the triangle for noisier W distributions than before.

In [15]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"],},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[1, 1, 1],
                            inflation_level_per_source=[2, 2, 2])
qtriangle_relax = InflationSDP(qtriangle)
cols = qtriangle_relax.build_columns('physical2', max_monomial_length=4)
qtriangle_relax.generate_relaxation(cols)

sdp, v_crit = bisection(qtriangle_relax, P_W_array, use_lpi_constraints=True)
v_crit

0.765045166015625

In [17]:
qtriangle_relax.set_distribution(P_W_array, use_lpi_constraints=True)
qtriangle_relax.solve(feas_as_optim=True, dualise=True)
print(qtriangle_relax.objective_value)

-0.037262938235857856


The critical value for the noise that we achieve, $v_{\text{crit}}=0.7650$, is lower than the critical value for the noise that we achieved in Example 7, $v_{\text{crit}}=0.8039$.

**Warning!** The tradeoff of using LPI constraints is that the dual certificate is no longer valid for other distributions. We can still certify incompatibility of a specific distribution $P_0$ with a certain causal model with the extracted certificate $\text{Poly}_{P_0}(P_0)>0$ when using LPI constraints, but when checking other distributions $P_1$ with the same certificate, satisfying he inequality $\text{Poly}_{P_0}(P_1)>0$ no longer guarantees that $P_1$ is also incompatible with the same causal structure.

# Partial information support

It is also interesting to study scenarios where not all the information about a particular distribution in the original scenario is known. Specifying particular elements of a distribution in an `InflationSDP` object is achieved via the use of the function `InflationSDP.set_values()`, which admits as input a dictionary where the keys are the variables to be assigned numerical quantities, and the corresponding values are the quantities themselves. 

## Example 9: Eavesdropped quantum repeater

An important example is the the analysis of cryptographic scenarios, where the honest parties may know their joint distribution but they cannot know their joint distribution together with a potential adversary. One simple such scenario is considered in Sec. VII in [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043). This scenario considers the quantum repeater/entanglement swapping experiment from Example 2 but with a hidden adversary, Eve, which is eavesdropping the sources $\rho_{AB}$ and $\rho_{BC}$ in an attempt to extract information about the secret key Alice and Charlie are trying to establish. Using quantum inflation, one can derive strict bounds on the amount of information Eve can extract about the secret key, as detailed in [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043). To implement this example with our package one would write:

In [ ]:
from examples_utils import EveGuessingProbability
import numpy as np
import itertools

InfProb = InflationProblem(dag={"rhoABE": ["A", "B", "E"],
                                "rhoBCE": ["B", "C", "E"]},
                           outcomes_per_party=[2, 4, 2, 2],
                           settings_per_party=[2, 1, 2, 1],
                           inflation_level_per_source=[2, 2])

InfSDP = InflationSDP(InfProb, commuting=False, verbose=1)

InfSDP.generate_relaxation(InfSDP.build_columns('local1', max_monomial_length=3))
meas = InfSDP.measurements  # accessed as meas[party][0][setting][outcome]

InfSDP.verbose = 0
visibilities, results = np.linspace(1, 0.85, 16), np.zeros(16)
for i, visibility in enumerate(visibilities):
    p = EveGuessingProbability(visibility)  # p[a,b,c,x,z] == p(a,b,c,x,z)
    p0 = np.sum(p[0,:,0,0,0])
    InfSDP.set_objective(meas[0][0][0][0]*meas[2][0][0][0]*meas[3][0][0][0]/p0
                         - meas[3][0][0][0])
    known_values = {}
    # 3 body terms
    for a, b, c, x, z in itertools.product(range(1), range(3), range(1), range(2), range(2)):
        known_values[meas[0][0][x][a]*meas[1][0][0][b]*meas[2][0][z][c]] = p[a,b,c,x,z]
    # 2 body terms
    for a, b, x in itertools.product(range(1), range(3), range(2)):
        known_values[meas[0][0][x][a]*meas[1][0][0][b]] = np.sum(p[a,b,:,x,0])
    for a, c, x, z in itertools.product(range(1), range(1), range(2), range(2)):
        known_values[meas[0][0][x][a]*meas[2][0][z][c]] = np.sum(p[a,:,c,x,z])
    for b, c, z in itertools.product(range(3), range(1), range(2)):
        known_values[meas[1][0][0][b]*meas[2][0][z][c]] = np.sum(p[:,b,c,0,z])
    # 1 body terms
    for a, x in itertools.product(range(1), range(2)):
        known_values[meas[0][0][x][a]] = np.sum(p[a,:,:,x,0])
    for b in range(3):
        known_values[meas[1][0][0][b]] = np.sum(p[:,b,:,0,0])
    for c, z in itertools.product(range(1), range(2)):
        known_values[meas[2][0][z][c]] = np.sum(p[:,:,c,0,z])

    InfSDP.set_values(known_values)
    #  InfSDP.solve()  # It takes several hours to solve
    print(visibility, InfSDP.objective_value)
    results[i] = InfSDP.objective_value

After running the code above and plotting the curve:
<center> <img src="./figures/repeater.PNG" alt="drawing" width="400"/> </center>

we recover the correct bounds from [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043).


## Example 10: Device independent entanglement certification

As mentioned in other sections, if all operators commute (by setting the `commuting` flag to `True` when instantiating `InflationSDP`), we get a relaxation that tests causal compatibility with a classical DAG. If furthermore the DAG is that of global shared randomness, then our package implements the techniques introduced in [Phys. Rev. X 7, 021042](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.021042). 

Testing compatibility of a distribution $p(abc\ldots | xyz\ldots)$ with a DAG with global shared randomness is the same as checking if the distribution $p(abc\ldots | xyz\ldots)$ is Bell-nonlocal. It is known that all distributions that are Bell-local form a set with the geometry of a polytope in the space of probability distributions. Linear programming techniques allow one to build an oracle that can tell whether a given distribution is inside or outside the polytope. See Section II from the [Bell nonlocality review](https://journals-aps-org.recursos.biblioteca.upc.edu/rmp/abstract/10.1103/RevModPhys.86.419) for more information. For many-body systems linear programming does not scale efficiently. The NPA hierarchy with all-to-all-commuting operators is an outer approximation of the linear programming method. In particular, the feasible set of the semidefinite programming relaxations strictly include the local set. However, given a sufficiently high level of the hierarchy, the feasible set becomes exactly the local set, i.e., it converges to the results of the linear program. For intermediate levels of the hierarchy, the approximation is less tight, but it is more efficiently implementable than the exact linear program.

In what follows, we reproduce a simple example from [Phys. Rev. X 7, 021042](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.021042) where we certify entanglement of the W state and the noise robustness of this technique. For this, we use tools from the [Hierarchy for nonlocality detection](https://github.com/FlavioBaccari/Hierarchy-for-nonlocality-detection) Github repository. This requires the use of the [QuTiP](https://qutip.org/) Python package to simulate measurements on the W state.

In [1]:
from causalinflation import InflationSDP, InflationProblem
from qutip import sigmax, sigmaz
from examples_utils import get_W_reduced, bisect_W

N = 7  # How many spins in the system
outcomes_per_party = [2] * N  # 2 measurements per site
settings_per_party = [2] * N  # 2 outcomes per site
InfProb = InflationProblem(outcomes_per_party=outcomes_per_party,
                           settings_per_party=settings_per_party, 
                           verbose=0)
InfSDP = InflationSDP(InfProb, commuting=True)
InfSDP.generate_relaxation('npa2')

W_state = get_W_reduced(N)
W_operators = [[[v.proj() for v in meas.eigenstates()[1]]
                          for meas in [sigmax(), sigmaz()]]
                          for p in range(N)]

bisect_W(InfSDP, W_state, W_operators, verbose=1)

0 Maximum smallest eigenvalue:    0.00911 	visibility = 0.5
1 Maximum smallest eigenvalue:  -0.001149 	visibility = 0.75
2 Maximum smallest eigenvalue:    0.00412 	visibility = 0.625
3 Maximum smallest eigenvalue:   0.001523 	visibility = 0.6875
4 Maximum smallest eigenvalue:  0.0001972 	visibility = 0.7188
5 Maximum smallest eigenvalue: -0.0004732 	visibility = 0.7344
6 Maximum smallest eigenvalue: -0.0001374 	visibility = 0.7266
7 Maximum smallest eigenvalue:  3.007e-05 	visibility = 0.7227
8 Maximum smallest eigenvalue:  -5.36e-05 	visibility = 0.7246
9 Maximum smallest eigenvalue: -1.176e-05 	visibility = 0.7236
10 Maximum smallest eigenvalue:  9.159e-06 	visibility = 0.7231
11 Maximum smallest eigenvalue: -1.298e-06 	visibility = 0.7234
12 Maximum smallest eigenvalue:  3.931e-06 	visibility = 0.7233
13 Maximum smallest eigenvalue:  1.317e-06 	visibility = 0.7233


We correctly recover the results for the W state visibility for $N=7$ from Table I in [Phys. Rev. X 7, 021042](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.021042).